In [14]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings

warnings.filterwarnings('ignore')

sp500_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
